# Kiesraad_AdviesScraper

Doel: Kiesraad adviezen uitgebracht vanaf 2019 tot en met 2024 verzamelen.

Toepassing: Bachelor-scriptie Informatiekunde, Universiteit van Amsterdam

Contact: mitchell.malaihollo@student.uva.nl

Metadata Formatering volgens de [Woogle metadata specificatie](https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema) 

In [1]:
import time
from urllib.parse import urljoin
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from openai import OpenAI
from dotenv import load_dotenv
import json
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urljoin, urlparse

In [2]:
# De Kiesraad volgt een andere paginatie methode als het AM en de ROB dus hier moet de laatste pagina handmatig worden toegevoegd
laatstepagina = 40

HREF_ID_WOORDEN = {"briefadvies", "wetsadvies", "advies", "adviesrapport", "adviesproject"}
ID_WOORDEN = {"briefadvies", "advies:", "advies", "adviesrapport", "adviesproject", "wetsadvies", "adviesvraag"}

USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) "
    "Gecko/20100101 Firefox/124.0 "
    "AdviescollegeScraper (+mailto:14605120@student.uva.nl; "
    "voor BSc-scriptie Informatiekunde UvA; "
    "doel: informatie inwinnen over verschillende adviescolleges)"
)

BASE_URL = (
    "https://www.kiesraad.nl/adviezen-en-publicaties?"
    "startdatum=01-01-2019"
    "&einddatum=31-12-2024"
)

env_path = r"C:\Users\31633\OneDrive\Bureaublad\OpenAI_API_Key.env"  
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
print("Key geladen:", api_key is not None)

Key geladen: True


In [5]:
import re

def beginSelenium():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={USER_AGENT}")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

driver = beginSelenium()
driver.get(BASE_URL)
time.sleep(2)
print("Maximaal aantal pagina's om te controleren: " + str(laatstepagina))
print("-----------------------")

adviesURLS = []
for paginaNummer in range(1, laatstepagina + 1):
    if paginaNummer == 1:
        BASE_paginaURL = BASE_URL
    else:
        BASE_paginaURL = f"{BASE_URL}&pagina={paginaNummer}"

    print("Pagina: " + str(paginaNummer))
    print(BASE_paginaURL)

    driver.get(BASE_paginaURL)
    time.sleep(2)

    li_items = driver.find_elements(
        By.CSS_SELECTOR,
        "ol.common.results li.results__item"
    )

    for li in li_items:
        try:
            a_tag = li.find_element(By.CSS_SELECTOR, "a.publication")
        except NoSuchElementException:
            continue
        
        href = a_tag.get_attribute("href")
        if href.startswith("/"):
            href = BASE + href

        # titel
        title_element = li.find_element(By.CSS_SELECTOR, "h3")
        title_text = title_element.text.strip().lower()

        # metadata
        meta_p = li.find_element(By.CSS_SELECTOR, "p.meta")
        meta_text = meta_p.text.strip().lower()
        doc_type = meta_text.split("|")[0].strip()

        # eerste woord van de href
        href_delen = urlparse(href).path
        href_deel = href_delen.strip("/").split("/")[-1]
        eerste_woord = href_deel.split("-")[0]

        # eerste woord van de titel
        eerste_woord_titel = title_text.split()[0] if title_text else ""

        # metadata check
        if "advies" in doc_type:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue

        # href eerste woord check
        if eerste_woord in HREF_ID_WOORDEN:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue

        # titel check
        if eerste_woord_titel in ID_WOORDEN:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue
    time.sleep(5)

    print("-------------------")

Maximaal aantal pagina's om te controleren: 40
-----------------------
Pagina: 1
https://www.kiesraad.nl/adviezen-en-publicaties?startdatum=01-01-2019&einddatum=31-12-2024
-------------------
Pagina: 2
https://www.kiesraad.nl/adviezen-en-publicaties?startdatum=01-01-2019&einddatum=31-12-2024&pagina=2
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/12/19/aanvullend-advies-over-het-vereenvoudigen-van-de-processen-verbaal-en-corrigenda
dit is een advies
-
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/11/12/advies-evaluatie-tijdelijke-experimentenwet-nieuwe-stembiljetten
dit is een advies
-
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/10/01/evaluatieadvies-ep-2024-voorspoedig-verlopen-verkiezing
dit is een advies
-
-------------------
Pagina: 3
https://www.kiesraad.nl/adviezen-en-publicaties?startdatum=01-01-2019&einddatum=31-12-2024&pagina=3
-------------------
Pagina: 4
https://www.kiesraad.nl/adviezen-en-publicaties?startdatum=01-01-2019&ei

In [6]:
print("\nAlle unieke adviezen links:")
for link in adviesURLS:
    print(link)

print(f"\nTotaal aantal unieke adviezen-links: {len(adviesURLS)}")


Alle unieke adviezen links:
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/12/19/aanvullend-advies-over-het-vereenvoudigen-van-de-processen-verbaal-en-corrigenda
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/11/12/advies-evaluatie-tijdelijke-experimentenwet-nieuwe-stembiljetten
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/10/01/evaluatieadvies-ep-2024-voorspoedig-verlopen-verkiezing
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/04/10/advies-wetsvoorstel-met-een-stem-meer-keus
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/04/04/advies-wet-verlenen-van-bijstand-bij-het-stemmen-in-een-stemhokje
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/03/13/evaluatie-van-de-tweede-kamerverkiezing-2023
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2023/12/19/advies-tijdelijke-experimentenregeling-nieuwe-stembiljetten
https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2023/11/09/advies-over

In [7]:
res = []
for url in adviesURLS:
    client = OpenAI()

    prompt = f"""
    Lees dit document: {url}

    Vul de onderstaande JSON in volgens https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema:

    {{
        "dc_title": "",
        "foi_publishedDate": "",
        "dc_description": ""
    }}

    Regels:
    - Alleen écht gevonden info invullen.
    - Max 100 woorden in dc_description.
    - Ontbreekt iets? dan "" laten.
    - datum in jjjj-mm-dd
    - Output = alleen geldige JSON niks anders.
    """
    response = client.responses.create(
        model="gpt-5.1",
        input=prompt,
        tools=[{"type": "web_search"}],
        max_output_tokens=800,
    )
    
    json_str = response.output_text.strip()

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Kon JSON niet parsen voor URL:", url)
        print("Ruwe output was:\n", json_str)
        continue

    data["dc_source"] = url
    data["dc_publisher"] = "zb000174"
    data["dc_type"] = "2e"
        
    res.append(data)
    
    print(json.dumps(data, ensure_ascii=False, indent=2))
    
    time.sleep(12)

with open("Kiesraad_adviesScraper_data1.json", "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)
    
print("\nAlle JSON-data opgeslagen in Kiesraad_adviesScraper_data1.json")

{
  "dc_title": "Aanvullend advies over het vereenvoudigen van de processen-verbaal en corrigenda",
  "foi_publishedDate": "2024-12-19",
  "dc_description": "De Kiesraad doet verbetervoorstellen voor 12 modellen die worden gebruikt door stembureaus, gemeentelijk stembureaus, hoofdstembureaus en centraal stembureaus. Het gaat om wijzigingen in samenvoegen van modellen en in vormgeving, opbouw en structuur. Dit advies volgt op het advies van 30 oktober 2023 over tekstuele aanpassingen in de modellen.",
  "dc_source": "https://www.kiesraad.nl/adviezen-en-publicaties/adviezen/2024/12/19/aanvullend-advies-over-het-vereenvoudigen-van-de-processen-verbaal-en-corrigenda",
  "dc_publisher": "zb000174",
  "dc_type": "2e"
}
{
  "dc_title": "Advies Evaluatie Tijdelijke experimentenwet nieuwe stembiljetten",
  "foi_publishedDate": "2024-11-12",
  "dc_description": "Tijdens de Europees Parlementsverkiezing van 6 juni 2024 is in vijf gemeenten geëxperimenteerd met een kleiner stembiljet op basis van 

In [8]:
with open("Kiesraad_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 59


In [9]:
res2 = []

with open("Kiesraad_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def retrieveLinksPagina2(driver, urltje):
    resje = {}
    driver.get(urltje)

    # Alle <li> elementen ophalen
    downloads = driver.find_elements(By.CSS_SELECTOR, "div.download a.download-chunk" )
    li_elements = driver.find_elements( By.CSS_SELECTOR, "div.block.docs-pubs.results ul.common li")
    
    for download in downloads:
        # Titel
        title = download.find_element( By.CSS_SELECTOR, "h2" ).text.replace("Download", "").strip().strip("'")
        
        # URL
        url = download.get_attribute("href")
    
        resje[title] = url

    for li in li_elements:
        # Titel
        title = li.find_element(By.CSS_SELECTOR,"h3").text.strip()
        
        # URL
        url = li.find_element(By.CSS_SELECTOR,"a.publication").get_attribute("href")

        resje[title] = url
    
    return resje    

driver = beginSelenium()

for item in data:
    dc_source2 = item.get("dc_source")

    urltjes = retrieveLinksPagina2(driver, dc_source2)

    foi_files = []
    for key, value in urltjes.items():
        if not value:
            continue
        foi_files.append({
            "foi_title": key,
            "foi_source": value,
        })

    res2.append({
        "dc_title": item.get("dc_title"),
        "foi_publishedDate": item.get("foi_publishedDate"),
        "dc_description": item.get("dc_description"),
        "dc_source": dc_source2,
        "dc_publisher": item.get("dc_publisher"),
        "dc_type": item.get("dc_type"),
        "foi_files": foi_files
    })

    time.sleep(5)

driver.quit()

with open("Kiesraad_adviesScraper_data2.json", "w", encoding="utf-8") as f:
    json.dump(res2, f, ensure_ascii=False, indent=2)

print("\nAlle JSON-data opgeslagen in Kiesraad_adviesScraper_data2.json")


Alle JSON-data opgeslagen in Kiesraad_adviesScraper_data2.json


In [10]:
with open("Kiesraad_adviesScraper_data2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 59
